In [ ]:
import openai
import os
import csv
import librosa
import numpy as np

def transcribe_audio(api_key, audio_file):
    client = openai.OpenAI(api_key=api_key)

    with open(audio_file, "rb") as file:
        response = client.audio.transcriptions.create(
            model="whisper-1",
            file=file,
            response_format="verbose_json"
        )

    return response

def compute_dropoff(audio_file):
    """Compute the degree of speaker’s voice gradually decreasing over an utterance."""
    y, sr = librosa.load(audio_file, sr=None)

    # Compute short-term energy over time
    frame_length = 2048
    hop_length = 512
    energy = np.array([
        np.sum(np.abs(y[i : i + frame_length]) ** 2)
        for i in range(0, len(y) - frame_length, hop_length)
    ])

    # Compute drop-off as the ratio of last energy value to first energy value
    dropoff_ratio = energy[-1] / energy[0] if energy[0] > 0 else 0
    return dropoff_ratio

def save_to_csv(results, output_file="/content/output_folder/output.csv"):
    """Save dropoff values to a CSV file."""
    os.makedirs("/content/output_folder", exist_ok=True)

    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["File Name", "Dropoff Ratio"])
        writer.writerows(results)

def process_multiple_files(api_key, audio_files, output_file):
    results = []

    for audio_file in audio_files:
        print(f"Processing {audio_file}...")
        transcribe_audio(api_key, audio_file)  # Using Whisper API
        dropoff_ratio = compute_dropoff(audio_file)
        results.append([os.path.basename(audio_file), dropoff_ratio])

    save_to_csv(results, output_file)
    print(f"All data saved to {output_file}")

if __name__ == "__main__":
    API_KEY = ""  # Replace with your OpenAI API key
    AUDIO_FILES = [
        "/content/ABB_segment_1.mp3",
        "/content/ABB_segment_2 (1).mp3",
        "/content/ABB_segment_3 (1).mp3"
    ]  # Replace with your list of audio files
    OUTPUT_FILE = "/content/output_folder/output.csv"

    process_multiple_files(API_KEY, AUDIO_FILES, OUTPUT_FILE)


Processing /content/ABB_segment_1.mp3...
Processing /content/ABB_segment_2 (1).mp3...
Processing /content/ABB_segment_3 (1).mp3...
All data saved to /content/output_folder/output.csv
